In [11]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [12]:
root = '/gdrive/My Drive/MSB481/'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# from keras.applications import MobileNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications import MobileNetV2


In [14]:
train_dir = os.path.join(root, "Garbage classification/")
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [15]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(384, 512, 3))
base_model.trainable = False ## Not trainable weights
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 384, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 192, 256, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 192, 256, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [16]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(6, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate = 0.0001), metrics=['accuracy'])

In [18]:
# creating generators for combining data and increasing the gainable insights by slightly modifying the images in the dataset

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(384, 512), batch_size=32, class_mode='binary', subset='validation')

Found 2032 images belonging to 6 classes.
Found 506 images belonging to 6 classes.


In [20]:
history = model.fit(train_generator, epochs=15, verbose=1, validation_data=validation_generator)

Epoch 1/15
64/64 [==============================] - 403s 6s/step - loss: 0.3440 - accuracy: 0.8775 - val_loss: 0.7395 - val_accuracy: 0.7352
Epoch 2/15
64/64 [==============================] - 402s 6s/step - loss: 0.3030 - accuracy: 0.8967 - val_loss: 0.7900 - val_accuracy: 0.7490
Epoch 3/15
64/64 [==============================] - 401s 6s/step - loss: 0.2904 - accuracy: 0.9011 - val_loss: 0.8093 - val_accuracy: 0.7372
Epoch 4/15
64/64 [==============================] - 402s 6s/step - loss: 0.2632 - accuracy: 0.9026 - val_loss: 0.8554 - val_accuracy: 0.7530
Epoch 5/15
64/64 [==============================] - 400s 6s/step - loss: 0.2418 - accuracy: 0.9168 - val_loss: 0.7862 - val_accuracy: 0.7688
Epoch 6/15
64/64 [==============================] - 403s 6s/step - loss: 0.2506 - accuracy: 0.9163 - val_loss: 0.8598 - val_accuracy: 0.7470
Epoch 7/15
64/64 [==============================] - 404s 6s/step - loss: 0.2332 - accuracy: 0.9139 - val_loss: 0.9348 - val_accuracy: 0.7431
Epoch 8/15
64

In [21]:
model.save_weights(os.path.join(root, "model/best_mobilenetv2.h5"))